In [126]:
#爬取--新浪财经/资产负债表
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 设置Selenium的Chrome浏览器选项
chrome_options = Options()
chrome_options.add_argument("--headless")  # 无头模式，不显示浏览器窗口
chrome_options.add_argument("--disable-gpu")  # 禁用GPU加速
chrome_options.add_argument("--no-sandbox")

# 指定chromedriver的路径（需提前下载对应版本的chromedriver并指定路径）
driver_path = 'path_to_chromedriver'

# 创建Chrome浏览器实例
driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)

# 初始化结果字典
result = {}

# 循环迭代年份
for year in range(2017, 2023):
    # 构建URL
    url = f"https://money.finance.sina.com.cn/corp/go.php/vFD_BalanceSheet/stockid/603486/ctrl/{year}/displaytype/4.phtml"

    # 访问目标网页
    driver.get(url)

    # 获取网页源代码
    html = driver.page_source

    # 使用BeautifulSoup解析网页内容
    soup = BeautifulSoup(html, 'html.parser')

    # 查找指定表格
    table_id = 'BalanceSheetNewTable0'
    table = soup.find('table', {'id': table_id})

    # 初始化要提取的表格行的标题
    desired_rows = ["流动资产合计", "非流动资产合计", "资产总计",
                    "流动负债合计", "非流动负债合计", "负债合计",
                    "所有者权益(或股东权益)合计", "负债和所有者权益(或股东权益)总计"]

     # 查找并提取指定表格行的数据
    rows = table.find_all('tr')
    df_year = pd.DataFrame()
    for row in rows:
        cells = row.find_all('td')
        if cells and cells[0].text.strip() in desired_rows:
            row_data = [cell.text.strip().replace(',', '') for cell in cells[1:]]
            df_year[cells[0].text.strip()] = row_data
    result[str(year)] = df_year
    # print(df_year)
# 关闭浏览器
driver.quit()

C:\Users\外星人\AppData\Local\Temp\ipykernel_18916\4180044449.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)


In [127]:
# 输出每年的DataFrame
for year, df in result.items():
    print(f"Year: {year}")
    # 合并每年的DataFrame并对每列求和
    print(df)
    print()

Year: 2017
      流动资产合计   非流动资产合计       资产总计     流动负债合计 非流动负债合计       负债合计  \
0  208564.88  61561.23  270126.11  143017.49  732.36  143749.85   
1  163494.46  58277.58  221772.04  116234.84  204.55  116439.39   

  所有者权益(或股东权益)合计 负债和所有者权益(或股东权益)总计  
0      126376.26         270126.11  
1      105332.65         221772.04  

Year: 2018
      流动资产合计   非流动资产合计       资产总计     流动负债合计  非流动负债合计       负债合计  \
0  336027.30  84784.71  420812.01  169876.00  1243.95  171119.95   
1  342685.50  76248.79  418934.28  187415.21  1611.50  189026.71   
2  311830.89  67976.99  379807.88  157172.81   551.80  157724.61   

  所有者权益(或股东权益)合计 负债和所有者权益(或股东权益)总计  
0      249692.06         420812.01  
1      229907.58         418934.28  
2      222083.27         379807.88  

Year: 2019
      流动资产合计    非流动资产合计       资产总计     流动负债合计  非流动负债合计       负债合计  \
0  316611.66  116597.43  433209.10  183240.19  1430.57  184670.76   
1  300510.68  113612.94  414123.62  166084.98  1393.86  167478.84   
2  295150.64  101175.13 

In [128]:
# 对每个年份的DataFrame进行求和
sum_data = {}
for year, df in result.items():
    print(year)
    filename = f"df_sum_{year}.xlsx"
    # 对合并后的DataFrame进行求和，并将字符串转换为浮点数类型
    df_sum = df.apply(lambda x: pd.to_numeric(x, errors='coerce')).sum()
    df_sum.to_excel(filename)
    print(df_sum)

2017
流动资产合计               372059.34
非流动资产合计              119838.81
资产总计                 491898.15
流动负债合计               259252.33
非流动负债合计                 936.91
负债合计                 260189.24
所有者权益(或股东权益)合计       231708.91
负债和所有者权益(或股东权益)总计    491898.15
dtype: float64
2018
流动资产合计                990543.69
非流动资产合计               229010.49
资产总计                 1219554.17
流动负债合计                514464.02
非流动负债合计                 3407.25
负债合计                  517871.27
所有者权益(或股东权益)合计        701682.91
负债和所有者权益(或股东权益)总计    1219554.17
dtype: float64
2019
流动资产合计               1208332.34
非流动资产合计               424729.36
资产总计                 1633061.71
流动负债合计                622405.56
非流动负债合计                 6105.76
负债合计                  628511.32
所有者权益(或股东权益)合计       1004550.40
负债和所有者权益(或股东权益)总计    1633061.71
dtype: float64
2020
流动资产合计               1484318.53
非流动资产合计               459017.46
资产总计                 1943335.99
流动负债合计                841739.63
非流动负债合计                 4088.05
负债合计           

In [129]:
import pandas as pd
# 定义要保留纵坐标的 Excel 文件
reference_file = 'df_sum_2017.xlsx'
# 定义要合并的其他 Excel 文件列表
file_list = ['df_sum_2018.xlsx', 'df_sum_2019.xlsx', 'df_sum_2020.xlsx', 'df_sum_2021.xlsx', 'df_sum_2022.xlsx']
# 读取参考文件的数据
merged_data = pd.read_excel(reference_file, index_col=0)
# 循环遍历其他文件，并将它们的数据与参考文件的索引匹配后合并
for file in file_list:
    data = pd.read_excel(file, index_col=0)
    merged_data = merged_data.merge(data, left_index=True, right_index=True)
# 重命名列名称为 "2017" 到 "2022"
merged_data.columns = [str(year) for year in range(2017, 2023)]
# 将合并后的数据保存到一个新的 Excel 文件
merged_data.to_excel('merged.xlsx')
print("合并完成！")

合并完成！


C:\Users\外星人\AppData\Local\Temp\ipykernel_18916\4237748595.py:11: FutureWarning: Passing 'suffixes' which cause duplicate columns {'0_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_data = merged_data.merge(data, left_index=True, right_index=True)
C:\Users\外星人\AppData\Local\Temp\ipykernel_18916\4237748595.py:11: FutureWarning: Passing 'suffixes' which cause duplicate columns {'0_x'} in the result is deprecated and will raise a MergeError in a future version.
  merged_data = merged_data.merge(data, left_index=True, right_index=True)


In [130]:
#数据计算
data = pd.read_excel('merged.xlsx')
data

,Unnamed: 0,2017,2018,2019,2020,2021,2022
0,流动资产合计,372059.34,990543.69,1208332.34,1484318.53,2749077.77,3957888.60
1,非流动资产合计,119838.81,229010.49,424729.36,459017.46,572218.61,767934.97
2,资产总计,491898.15,1219554.17,1633061.71,1943335.99,3321296.38,4725823.56
3,流动负债合计,259252.33,514464.02,622405.56,841739.63,1569108.23,1946676.66
4,非流动负债合计,936.91,3407.25,6105.76,4088.05,101451.58,413602.98
5,负债合计,260189.24,517871.27,628511.32,845827.68,1670559.82,2360279.64
6,所有者权益(或股东权益)合计,231708.91,701682.91,1004550.40,1097508.33,1650736.57,2365543.92
7,负债和所有者权益(或股东权益)总计,491898.15,1219554.17,1633061.71,1943335.99,3321296.38,4725823.56


In [131]:
# 转置 DataFrame
df_transposed = data.transpose()
df_transposed
# 删除指定行
df_transposed = df_transposed.drop('Unnamed: 0')
df_transposed
# 设置正确的列名
df_transposed.columns = ["流动资产合计", "非流动资产合计", "资产总计", "流动负债合计", "非流动负债合计", "负债合计", "所有者权益(或股东权益)合计", "负债和所有者权益(或股东权益)总计"]
df_transposed

,流动资产合计,非流动资产合计,资产总计,流动负债合计,非流动负债合计,负债合计,所有者权益(或股东权益)合计,负债和所有者权益(或股东权益)总计
2017,372059.34,119838.81,491898.15,259252.33,936.91,260189.24,231708.91,491898.15
2018,990543.69,229010.49,1219554.17,514464.02,3407.25,517871.27,701682.91,1219554.17
2019,1208332.34,424729.36,1633061.71,622405.56,6105.76,628511.32,1004550.4,1633061.71
2020,1484318.53,459017.46,1943335.99,841739.63,4088.05,845827.68,1097508.33,1943335.99
2021,2749077.77,572218.61,3321296.38,1569108.23,101451.58,1670559.82,1650736.57,3321296.38
2022,3957888.6,767934.97,4725823.56,1946676.66,413602.98,2360279.64,2365543.92,4725823.56


In [132]:
# 将相关列转换为数值类型
df_transposed[['流动资产合计', '非流动资产合计', '资产总计','流动负债合计','非流动负债合计','负债合计','所有者权益(或股东权益)合计','负债和所有者权益(或股东权益)总计']] = df_transposed[['流动资产合计', '非流动资产合计', '资产总计','流动负债合计','非流动负债合计','负债合计','所有者权益(或股东权益)合计','负债和所有者权益(或股东权益)总计']].astype(float)

In [133]:
# 计算流动资产占比和非流动资产占比
df_transposed['流动资产占比'] = df_transposed['流动资产合计'] / df_transposed['资产总计']
df_transposed['非流动资产占比'] = df_transposed['非流动资产合计'] / df_transposed['资产总计']
# 计算流动负债占比、非流动负债占比和所有者权益占比
df_transposed['流动负债占比'] = df_transposed['流动负债合计'] / df_transposed['负债合计']
df_transposed['非流动负债占比'] = df_transposed['非流动负债合计'] / df_transposed['负债合计']
df_transposed['所有者权益占比'] = df_transposed['所有者权益(或股东权益)合计'] / df_transposed['负债和所有者权益(或股东权益)总计']

In [134]:
df_transposed

,流动资产合计,非流动资产合计,资产总计,流动负债合计,非流动负债合计,负债合计,所有者权益(或股东权益)合计,负债和所有者权益(或股东权益)总计,流动资产占比,非流动资产占比,流动负债占比,非流动负债占比,所有者权益占比
2017,372059.34,119838.81,491898.15,259252.33,936.91,260189.24,231708.91,491898.15,0.756375,0.243625,0.996399,0.003601,0.471051
2018,990543.69,229010.49,1219554.17,514464.02,3407.25,517871.27,701682.91,1219554.17,0.812218,0.187782,0.993421,0.006579,0.575360
2019,1208332.34,424729.36,1633061.71,622405.56,6105.76,628511.32,1004550.40,1633061.71,0.739918,0.260082,0.990285,0.009715,0.615133
2020,1484318.53,459017.46,1943335.99,841739.63,4088.05,845827.68,1097508.33,1943335.99,0.763799,0.236201,0.995167,0.004833,0.564755
2021,2749077.77,572218.61,3321296.38,1569108.23,101451.58,1670559.82,1650736.57,3321296.38,0.827712,0.172288,0.939271,0.060729,0.497016
2022,3957888.60,767934.97,4725823.56,1946676.66,413602.98,2360279.64,2365543.92,4725823.56,0.837502,0.162498,0.824765,0.175235,0.500557


In [135]:
# 选择需要显示的列
df_result = df_transposed[['流动资产占比', '非流动资产占比']]
df_result

,流动资产占比,非流动资产占比
2017,0.756375,0.243625
2018,0.812218,0.187782
2019,0.739918,0.260082
2020,0.763799,0.236201
2021,0.827712,0.172288
2022,0.837502,0.162498


In [136]:
# 负债和所有者权益结构
df_result1 = df_transposed[['流动负债占比', '非流动负债占比', '所有者权益占比']]
df_result1

,流动负债占比,非流动负债占比,所有者权益占比
2017,0.996399,0.003601,0.471051
2018,0.993421,0.006579,0.575360
2019,0.990285,0.009715,0.615133
2020,0.995167,0.004833,0.564755
2021,0.939271,0.060729,0.497016
2022,0.824765,0.175235,0.500557


In [137]:
#爬取利润表
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

base_url = "https://money.finance.sina.com.cn/corp/go.php/vFD_ProfitStatement/stockid/603486/ctrl/{}/displaytype/4.phtml"

# 创建一个Excel工作簿
workbook = Workbook()

# 遍历2017年至2022年的数据
for year in range(2017, 2023):
    url = base_url.format(year)

    # 发送GET请求获取网页内容
    response = requests.get(url)

    # 解析网页内容
    soup = BeautifulSoup(response.text, "html.parser")

    # 找到目标表格
    table = soup.find("table", id="ProfitStatementNewTable0")

    # 提取表格中的数据
    rows = table.find_all("tr")

    # 创建一个工作表，以年份命名
    sheet = workbook.create_sheet(str(year))

    # 遍历表格的行和列，将数据写入工作表
    for row in rows:
        cells = row.find_all("td")
        row_data = [cell.text.strip() for cell in cells]

        # 判断是否为目标行数据
        if len(row_data) > 0 and ("营业收入" in row_data[0] or "营业利润" in row_data[0] or "报表日期" in row_data[0]):
            sheet.append(row_data)

# 删除默认创建的工作表
workbook.remove(workbook["Sheet"])

# 保存Excel文件
workbook.save("data_2017_2022.xlsx")

In [138]:
import pandas as pd

# 从Excel文件读取数据并创建DataFrame
df = pd.read_excel("data_2017_2022.xlsx", sheet_name=None)

# 创建一个空列表，用于存储处理后的数据
new_dfs = []

# 遍历每个工作表
for sheet_name, sheet_df in df.items():
    # 选择营业收入和营业利润所在的行
    revenue_row = sheet_df.iloc[0]
    profit_row = sheet_df.iloc[1]

    # 提取日期列和营业收入、营业利润列
    dates = revenue_row.index[1:]
    revenue_values = revenue_row.values[1:]
    profit_values = profit_row.values[1:]

    # 将营业收入和营业利润转换为float并相加
    revenue_sum = sum([float(value.replace(",", "")) for value in revenue_values])
    profit_sum = sum([float(value.replace(",", "")) for value in profit_values])

    # 创建新的DataFrame
    new_df = pd.DataFrame({
        "日期": dates,
        "营业收入总和": [revenue_sum] * len(dates),
        "营业利润总和": [profit_sum] * len(dates)
    })

    # 保留第一行数据
    new_df = new_df.iloc[:1, :]

    # 将日期改为工作表的名称
    new_df["日期"] = sheet_name

    # 将新的DataFrame添加到列表中
    new_dfs.append(new_df)

# 将所有处理后的数据合并为一个DataFrame
final_df = pd.concat(new_dfs)

# 打印最终的DataFrame
print(final_df)

     日期      营业收入总和     营业利润总和
0  2017   944358.52   91191.66
0  2018  1317879.76  125747.34
0  2019  1243209.47   49246.68
0  2020  1474053.55  122682.01
0  2021  2891367.15  511917.15
0  2022  3547301.67  448291.00


In [139]:
# 计算营业收入的环比增长率
final_df["营业收入环比增长率"] = final_df["营业收入总和"].pct_change() * 100

# 计算营业利润的环比增长率
final_df["营业利润环比增长率"] = final_df["营业利润总和"].pct_change() * 100

# 打印计算后的DataFrame
final_df

,日期,营业收入总和,营业利润总和,营业收入环比增长率,营业利润环比增长率
0,2017,944358.52,91191.66,NaN,NaN
0,2018,1317879.76,125747.34,39.552906,37.893465
0,2019,1243209.47,49246.68,-5.665941,-60.836802
0,2020,1474053.55,122682.01,18.568398,149.117321
0,2021,2891367.15,511917.15,96.150754,317.271571
0,2022,3547301.67,448291.00,22.685964,-12.428994


In [140]:
df_result2 = final_df[['日期','营业收入环比增长率', '营业利润环比增长率']]
df_result2.dropna()

,日期,营业收入环比增长率,营业利润环比增长率
0,2018,39.552906,37.893465
0,2019,-5.665941,-60.836802
0,2020,18.568398,149.117321
0,2021,96.150754,317.271571
0,2022,22.685964,-12.428994
